# 1. Реализовать функцию, аналогичную встроенной функции reverse!, назвав её, например, reverse_user!, для следующих случаев: a) аргумент функции - вектор б) аргумент функции - матрица (2-мерный массив)

In [6]:
function reverse_user!(A::Vector)
    for i in 1:length(A) / 2
        A[i], A[length(A) - i + 1] = A[length(A) - i + 1], A[i]
    end
end

reverse_user! (generic function with 1 method)

In [7]:
function reverse_user!(A::Matrix)
    dimensions = ndims(A)
    A = reshape(A, length(A))
    reverse_user!(A)
    A = reshape(A, dimensions)
end

reverse_user! (generic function with 2 methods)

# 2. Аналогично, реализовать функцию, аналогичную встроенной функции copy для следующих случаев: a) аргумент функции - вектор б) аргумент функции - матрица (2-мерный массив)

In [8]:
function copy_user(A::Vector{Any})
    Copy = Vector{typeof(A)}(undef, length(A))
    for i in 1:length(A)
        Copy[i] = A[i]
    end
    return Copy
end

copy_user (generic function with 1 method)

In [9]:
function copy_user(A::Matrix)
    Copy = Matrix{typeof(A)}(undef, size(A))
    for i in 1:size(A, 2)
        Copy[:,i] = @view A[:,i]
    end
    return Copy
end

copy_user (generic function with 2 methods)

# 3. Реализовать алгоритм сортировки методом пузырька, написав следующие 4 обобщенные функции: bubblesort, bubblesort!, bubblesortperm, bubblesortperm!, по аналогии со встоенными функциями sort!, sort, sortperm!, sortperm, ограничившись только случаем, когда входной параметр есть одномерный массив (вектор).

In [10]:
function bubblesort!(A)
    n = length(A)
    for k in 1:n-1
        istranspose = false
        for i in 1:n-k
            if A[i]>A[i+1]
                A[i], A[i+1] = A[i+1], A[i]
                istranspose = true
            end
        end
        if istranspose == false
            break
        end
    end
    return A
end

bubblesort! (generic function with 1 method)

In [11]:
bubblesort(A)=bubblesort!(deepcopy(A))

bubblesort (generic function with 1 method)

In [12]:
function bubblesortperm!(A)
    n = length(A)
    indexes = collect(1:n)
    for k in 1:n-1
        istranspose = false
        for i in 1:n-k
            if A[i]>A[i+1]
                A[i],A[i+1]=A[i+1],A[i]
                indexes[i],indexes[i+1]=indexes[i+1],indexes[i]
                istranspose = true
            end
        end
        if istranspose == false
            break
        end
    end
    return indexes
end

bubblesortperm! (generic function with 1 method)

In [13]:
bubblesortperm(A)=bubblesortperm!(deepcopy(A))

bubblesortperm (generic function with 1 method)

# 4. На основе разработанных в пункте 1 функций, сотрирующих одномерный массив, написать соответствующие функции, которые бы могли получать на вход матрицу, и сортировать каждый из ее столбцов по отдельности. Имена функций оставить прежними, что были и в пункте 1, воспользовавшись механизмом множественной диспетчеризации языка Julia.

In [14]:
function bubblesort!(A::Matrix)
    _ , column_size = size(A)
    for i in 1:column_size
        column = @view A[:, i]
        bubblesort!(column)
    end
end

bubblesort(A::Matrix) = bubblesort!(copy(A))

bubblesort (generic function with 2 methods)

In [15]:
function bubblesortperm!(A::Matrix)
    row_size , column_size = size(A)
    B = copy(A)
    k = 0
    for i in 1:column_size
        column = @view A[:, i]
        index_array = bubblesortperm!(column)
        index_column = @view B[: , i]
        for j in 1:row_size
            index_column[j] = index_array[j] + k
        end
        k += 2
    end

    return B
end

bubblesortperm(A::Matrix) = bubblesortperm!(copy(A))

bubblesortperm (generic function with 2 methods)

# 5. Написать функцию sortkey!(a, key_values), получающую на вход некоторый вектор a, и соответствующий вектор keyvalues ключевых значений элементов вектора a, осуществляющую сортировку вектора a по ключевым значениям его элементов, и возвращающую ссылку на вектор a. (Для сортировки вектора ключевых значений можно востпользоваться одной из разработанных в пункте 1 функций, или соответствующей встроенной функцией).

In [16]:
function sortkey!(A, key_values)
    num = bubblesortperm!(key_values)
    return @view A[num]
end

function sortkey!(A, F::Function)
    num = bubblesortperm!(F(A))
    return A[num]
end

sortkey! (generic function with 2 methods)

# 6. Написать функцию calcsort, реализующую сортировку методом подсчета числа значений. Рассмотреть 2 варианта функции (2 метода - в терминологии Julia): в первом варианте возможные значения элементов сортируемого массива задаются некоторым диапазоном, во втором - некоторым отсортированным массивом (вектором).

In [17]:
function calcsort!(A,values::UnitRange{Int})
    store = zeros(Int, size(values))
    for i in 1:length(A)
        for j in 1:length(values)
            if A[i]==values[j]
                store[j]+=1
            end
        end
    end
    k=1
    for i in 1:length(values)
        while store[i]>0
            A[k]=values[i]
            k+=1
            store[i]-=1
        end
    end
    return A
end

function calcsort!(A,values::Vector{Int})
    store = zeros(Int, size(values))
    for i in 1:length(A)
        for j in 1:length(values)
            if A[i]==values[j]
                store[j]+=1
            end
        end
    end
    k=1
    for i in 1:length(values)
        while store[i]>0
            A[k]=values[i]
            k+=1
            store[i]-=1
        end
    end
    return A
end

calcsort! (generic function with 2 methods)

# 7. Написать функции insertsort!, insertsort, insertsortperm, insertsortperm! (по аналогии с пунктом 1) реализующие алгоритм сортировки вставками

In [18]:
function insertsort!(A)
    for i in 2:length(A)
        key = A[i]
        j = i - 1
        while j >= 1 && A[j] > key
            A[j + 1] = A[j]
            j -= 1
        end
        A[j + 1] = key
    end
    return A
end

function insertsort(A)
    insertsort!(deepcopy(A))
end

function insertsortperm!(A)
    numbers = collect(1:length(A))
    for i in 2:length(A)
        key = A[i]
        j = i - 1
        while j >= 1 && A[j] > key
            A[j + 1] = A[j]
            numbers[j + 1],numbers[j] =numbers[j],numbers[j + 1]
            j -= 1
        end
        A[j + 1] = key
        numbers[j + 1],numbers[i] = numbers[i],numbers[j + 1]
    end
    return numbers
end

function insertsortperm(A)
    insertsortperm!(deepcopy(A))
end

insertsortperm (generic function with 1 method)

# 8. Реализовать ранее написанную функцию insertsort! с помощью встроенной функции reduce.

In [19]:
insertsort!(A)=reduce(1:length(A))do _, k
    while k>1 && A[k-1] > A[k]
        A[k-1], A[k] = A[k], A[k-1]
        k-=1
    end
    return A
end

insertsort! (generic function with 1 method)

# 9. Дополнить функцию insrtsort! процедурой "быстрого поиска".

In [20]:
function quicsearch(A,b)
    first = firstindex(A)
    last = lastindex(A)
    index = -1
    bool= false
    while (first+1 < last) &&(index == -1)
        mid = (first+last)÷2
        if A[mid] == b
            index = mid
            bool=true
        else
            if b<A[mid]
                last = mid -1
            else
                first = mid +1
            end
        end
    end
    if A[first]>b
        index=first
        bool=false
    elseif A[last]>b
        index=last
        bool=false
    elseif A[last]<b
        index=last+1
        bool=false
    end
    return bool,index
end

function insertsort!(A)
    for i in 2:length(A)
        key = A[i]
        j = i
        c,d=quicsearch(A[1:i-1],key)
        while d!=j 
            A[j],A[j-1]=A[j-1],A[j]
            j=j-1
        end
    end
    return A
end

insertsort! (generic function with 1 method)

# 10. Написать обобщенную функцию (nummax), получающую на вход итерируемый объект, содержащий некоторую последовательность (элементы которой можно сравнивать по величине), и возвращающую число максимумов этой последовательности.

In [21]:
function nummax(A)
    max= maximum(A)
    count = 1
    for i in firstindex(A)+1:lastindex(A)
        if A[i] == max
            count+=1
        end
    end
    return count
end

nummax (generic function with 1 method)

# 11. Написать обобщенную функцию (findallmax), получающую на вход итерируемый объект, содержащий некоторую последовательность (элементы которой можно сравнивать по величине), и возвращающую вектор, составленный из индексов элементов входной последовательности, имеющих максимальное значение.

In [22]:
function findallmax(A)
    max = maximum(A)
    count = nummax(A)
    result = Vector{Int}(undef,count)
    for i in firstindex(A)+1:lastindex(A)
        if A[i]==max   
            push!(result, i)
        end
    end
    return result
end

findallmax (generic function with 2 methods)

# 12. Написать обобщенную функцию findallmax высшего порядка, получающую на вход итерируемый объект, содержащий некоторую последовательность и некоторую функцию (значение типа ::Function), и возвращающую вектор, составленный из индексов элементов входной последовательности, на которых заданная функция достигает максимального значения (речь и дет о сужении заданной функции на заданной последовательности).

In [23]:
function findallmax(A, func::Function)
    B = copy(A)
        for i in 1:length(B)
            B[i] = func(B[i])
        end
    return findallmax(B)
end

findallmax (generic function with 2 methods)